In [1]:
import win32com.client
import os
import re
from photoshop import Session
import photoshop.api as ps
import keyboard
import time
import subprocess
from tqdm import tqdm
import cv2
import pyautogui
import numpy as np

In [2]:
app = win32com.client.Dispatch('Photoshop.Application')
app.Open(r"C:\Users\dahom\Desktop\ps_screen\output\Screens_640\Screens_PSD\DCC002.psd")
doc = app.Application.ActiveDocument
app.DisplayDialogs = 3

In [3]:
allLayers=[]
def getAllLayers(doc, allLayers):
    for i in range(0, len(doc.Layers)):
        layer = doc.Layers[i]
        if(layer.typename == 'ArtLayer'):
            allLayers.append(layer)
        else:
            getAllLayers(layer, allLayers)
    return allLayers

In [17]:
def editButton(num, newText):
    # edit a specific buttons text
    lang = getLang(newText)
    #print('btn{}.lang({})'.format(str(num), lang))
    layer = doc.LayerSets("Buttons").Layers("btn"+str(num)).Layers(lang)
    layer.visible = True
    text_of_layer = layer.TextItem
    text_of_layer.contents = newText
    #  hide the other text?
    doc.LayerSets("Buttons").Layers("btn"+str(num)).Layers('ara' if lang == 'eng' else 'eng').visible = False


In [18]:
# AddText, disable else text
def createText(text, pos):
    lang = getLang(text)
    layer = doc.LayerSets("Buttons").Layers("floatingText")
    layer = doc.ArtLayers.Add()
    layer.kind = 2 #TEXT
    layer.textItem.contents = text
    layer.textItem.position = [pos[0], pos[1]]
    layer.textItem.size = 13 if lang == 'eng' else 14
    layer.textItem.Font = 'AvantGardeITCbyBT-Medium' if lang == 'eng' else 'GESSTextMedium-Medium'
    layer.textItem.Justification = 3 if lang == 'eng' else 1
    # Color???
    layer.Move(doc.LayerSets("Buttons").LayerSets("floatingText"),1)
    return layer


In [8]:
# filename = 'ICON257.jpg'
# mngText(filename)
# mngButtons(filename)

In [4]:
def saveTo(path, method=2):
    if(method == 0): #PSD
#         doc.SaveAs(SaveIn=path, AsCopy=True, ExtensionType=2)
        options = win32com.client.Dispatch('Photoshop.PhotoshopSaveOptions')
        options.embedColorProfile = True;
        options.alphaChannels = True;
        doc.SaveAs(SaveIn=path, AsCopy=False, Options=options, ExtensionType=2)
    elif(method == 1):
        options = win32com.client.Dispatch('Photoshop.PNGSaveOptions')
        doc.SaveAs(SaveIn=path, Options=options, AsCopy=True, ExtensionType=2)
    else:
        # Very small Size compared to SaveAs
        options = win32com.client.Dispatch('Photoshop.ExportOptionsSaveForWeb')
        options.Format = 13
        options.PNG8 = False
        doc.Export(ExportIn=path, ExportAs=2, Options=options)

In [20]:
def do_action(filename):
    filePath = os.path.join(input_dir, filename)
    outFilePath = os.path.join(output_dir, filename)
    mngText(filePath)
    mngButtons(filePath)
   

In [54]:
# def needAction():
#     image = pyautogui.screenshot()
#     image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
#     for x in range(image.shape[0]):
#         for y in range(image.shape[1]):
#             b, g, r = image[x, y]
#             if((r, g, b) == (0, 139, 147)):
#                 return True
#     return False

In [5]:
def rasterizeProblematic(allLayers):
    problematic = ["Vector Smart Object copy", "Vector Smart Object"]
    for layer in allLayers:
        if(layer.Kind == 17 and layer.name in problematic):
#             print(f'\t rasterizing {layer.name}')
            layer.Rasterize(6)

In [8]:
# NEXT 183 
ctr = 0
# Detect red, save png psd, resize, save png psd
allLayers=[]
input_dir = r'C:\Users\dahom\Desktop\ps_screen\output\Screens_640\Screens_PSD'
output1_dir = r"C:\Users\dahom\Desktop\ps_screen\changeRedToCyan\output_640"
output2_dir = r"C:\Users\dahom\Desktop\ps_screen\changeRedToCyan\output_800"
for root, directories, files in os.walk(input_dir, topdown=False):
    for name in tqdm(files):
        ctr += 1
        if(ctr < 183): 
            continue
        
        file_path = os.path.join(root, name)
        noextName = os.path.splitext(name)[0]
        app.Open(file_path)
        doc = app.Application.ActiveDocument
        save = False
        
        layers = getAllLayers(doc, allLayers)
        for layer in layers:
            if (layer.kind == 2 and layer.visible and layer.textItem.color.rgb.hexValue == 'E30613'):
                    solidColor = win32com.client.Dispatch('Photoshop.SolidColor')
                    solidColor.rgb.hexValue = "008B93"
                    layer.textItem.color = solidColor                    
                    save = True
        
        if(save):
            saveTo(os.path.join(output1_dir, noextName), 1) # save to output
            saveTo(os.path.join(output1_dir, name),0) # save to output
#         Resize and Save
            rasterizeProblematic(layers)
            doc.ResizeImage(800, 600, 150, 8)
            saveTo(os.path.join(output2_dir, noextName), 1) # save to output
            saveTo(os.path.join(output2_dir, name),0) # save to output
        doc.Close(2)



100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [26:26<00:00,  5.67s/it]
